# 集装箱直接与缓冲位交付模式下AGV与ASC集成调度优化

# Scheduling ASC and AGV considering direct and buffer-based modes for transferring containers


# 1 问题描述

在自动化集装箱码头中，自动化导引车(automated guided vehicle, AGV)与自动化堆垛机(automated stacking crane, ASC)分别完成水平运输与堆场垂直装卸作业。

自动化堆场的海侧交接区配置直接和缓冲位交付工艺，用于集装箱的直接或间接(以缓冲位作为中介)交付作业。在缓冲位交付模式下，ASC与AGV之间通过缓冲设备进行集装箱的暂存和交付。

缓冲位交付模式的应用，引入缓冲设备解耦ASC与AGV的作业工序，缓解了设备之间相互等待的现象，在减小等待时间的同时提高了AGV的利用率。然而，受制于堆场两端的空间，缓冲设备数量有限，在作业繁忙时允许集装箱暂存在缓冲设备中的时间较短，集装箱必须尽快移出缓冲设备，否则将导致等待而降低作业效率。

在随动控制技术的前提下，ASC与AGV两类设备的动态定位与控制成为可能。一些码头尝试使用ASC与AGV之间的集装箱直接交付模式。具体而言，在交付作业时，作业设备根据控制指令不断调整他们的位置，最终实现直接接触作业——ASC直接把出口集装箱卸载(drop)到AGV上或直接抓取(lift)AGV上的进口集装箱，实现集装箱在海侧交接区“即来即走”的作业模式。


# 2 调度优化模型

In [1]:
# -*- coding: utf-8 -*-
import time
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import math as mt
import matplotlib.pyplot as plt
import time

In [2]:
mDir = '../data/'
mDir_input = 'input/'
mDir_solution = 'solution/'

In [3]:
def data_generation(N, NK, NA, NB, NAME, AGV, ASC):
    dataset = pd.read_excel(mDir + mDir_input + 'AGV-ASC flexiable interaction.xlsx', 
                            sheet_name=NAME, skiprows=0)
    df = pd.DataFrame(dataset)
    dataExp = df.loc[1: 7, ['No', 'type', 'XO', 'YO', 'XD', 'YD']]
    
    J = list(df['No'].values)[1:N+1]
    TP = list(df['type'].values)[0:N+1]
    XO = list(df['XO'].values)[0:N+1]
    YO = list(df['YO'].values)[0:N+1]
    XD = list(df['XD'].values)[0:N+1]
    YD = list(df['YD'].values)[0:N+1]

    JS = list(range(0, N+1))
    JE = list(range(1, N+2))
    JD = list(range(0, N+2))

    K = set(list(range(1, NK+1)))
    B = set(list(range(1, NB+1)))
    A = set(list(range(1, NA+1)))

    JI = [i for i, x in enumerate(TP) if x == 1]        # 进口箱
    JO = [j for j, x in enumerate(TP) if x == 0]        # 出口箱

    # AGV = {'SPD': 4, 'BK': 10, 'TQ': 15}        # AGV参数：水平移动速度、支架上抬/放箱时间、岸桥下作业时间
    # ASC = {'SPD': 4, 'BY': 5, 'TY': 10}        # ASC参数：水平移动速度、抓取/释放箱时间、起/落吊时间
    Container = {'Len': 12, 'Wdh': 3}           # 集装箱规格：长、宽

    LocB = (200, 215)        # 缓冲区B的位置坐标(x,y)
    BayB = (0, 5)            # 缓冲区B在箱区中的位置(贝位，列位)
    TTKN = np.zeros([N+1, 1])
    TTYN = np.zeros([N+1, 1])
    TTK, TTY = [], []
    for i in J:
        if i in JI:
            TTKN[i][0] = (1/AGV['SPD']) * (abs(XO[i] - LocB[0]) + abs(YO[i] - LocB[1]))
        elif i in JO:
            TTKN[i][0] = (1/AGV['SPD']) * (abs(XD[i] - LocB[0]) + abs(YD[i] - LocB[1]))

    for i in J:
        if i in JI:
            TTYN[i][0] = (1/ASC['SPD']) * ((XD[i] - BayB[0]) * Container['Len'] + abs(YD[i] - BayB[1]) * Container['Wdh'])
        elif i in JO:
            TTYN[i][0] = (1/ASC['SPD']) * ((XO[i] - BayB[0]) * Container['Len'] + abs(YO[i] - BayB[1]) * Container['Wdh'])

    for j in range(N+1):
        TTK.append(float(TTKN[j][0]))
        TTY.append(float(TTYN[j][0]))

    TK, TY = np.zeros([N+2, N+2]), np.zeros([N+2, N+2])

    for a in range(N+2):
        for b in range(N+2):
            if a in JI:
                if b in JI:
                    TK[a][b] = (1/AGV['SPD']) * (abs(LocB[0] - XO[b]) + abs(LocB[1] - YO[b]))
                    TY[a][b] = (1/ASC['SPD']) * (abs(XD[a] - BayB[0]) * Container['Len'] + abs(YD[a] - BayB[1]) * Container['Wdh'])
                elif b in JO:
                    # TK[a][b] = (1/AGV['SPD']) * (abs(LocB[0] - XD[b]) + abs(LocB[1] - YD[b]))
                    TK[a][b] = 0
                    TY[a][b] = (1/ASC['SPD']) * (abs(XD[a] - XO[b]) * Container['Len'] + abs(YD[a] - YO[b]) * Container['Wdh'])
            elif a in JO:
                if b in JI:
                    TK[a][b] = (1/AGV['SPD']) * (abs(XD[a] - XO[b]) + abs(YD[a] - YO[b]))
                    TY[a][b] = (1/ASC['SPD']) * (abs(BayB[0] - XD[b]) * Container['Len'] + abs(BayB[1] - YD[b]) * Container['Wdh'])
                elif b in JO:
                    TK[a][b] = (1/AGV['SPD']) * (abs(XD[a] - LocB[0]) + abs(YD[a] - LocB[1]))
                    TY[a][b] = (1/ASC['SPD']) * (abs(BayB[0] - XO[b]) * Container['Len'] + abs(BayB[1] - YO[b]) * Container['Wdh'])

    for a in range(N+2):
        for b in range(N+2):
            if a == 0:
                TK[a][b] = 0
                TY[a][b] = 0
            if b == 0:
                TK[a][b] = 9999
                TY[a][b] = 9999
            if b == N+1:
                TK[a][b] = 0
                TY[a][b] = 0
            if a == N+1:
                TK[a][b] = 9999
                TY[a][b] = 9999
    TK[0][N+1] = 9999
    TY[0][N+1] = 9999

    return J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, dataExp

In [4]:
N = 8    # 任务数量
NK = 4   # AGV数量
NA = 2   # 直接交付位数量
NB = 3   # 缓冲交付位数量

name = "testing"
AGV = {'SPD': 4, 'BK': 10, 'TQ': 15}  # AGV参数：水平移动速度、支架上抬/放箱时间、岸桥下作业时间
ASC = {'SPD': 6, 'BY': 5, 'TY': 10}  # ASC参数：水平移动速度、抓取/释放箱时间、起/落吊时间

J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, dataExp = data_generation(N, NK, NA, NB, name, AGV, ASC)
print('数据展示：\n[任务编号(No)，任务类型(Type)，起始贝位(XO)，起始贝位(YO), 目标贝位(XD)，目标贝位(YD)]\n')
print(dataExp)

数据展示：
[任务编号(No)，任务类型(Type)，起始贝位(XO)，起始贝位(YO), 目标贝位(XD)，目标贝位(YD)]

   No  type  XO   YO  XD   YD
1   1     0   1   10   0  100
2   2     1   0  200  18   10
3   3     1   0  300  35    6
4   4     0   7    1   0  300
5   5     1   0  100  37    1
6   6     1   0  200  27    8
7   7     0  17    9   0  300


## 2.1 直接交付模式下集成调度优化模型 [M1]

In [21]:
def modelASolvebyCplex(N, J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, Gap, TimeLimit):
    tour_K, tour_Y = [], []
    t_start = time.time()
    
    # define Model
    m = Model("AGV_ASC_flexibleInteractionScheduling")
    S = 0
    E = N+1
    M1, M2, M3, M4 = 1e+4, 1e+4, 1e+5, 1e+5

    # define Variables
    x = m.binary_var_cube(len(K) + 1, N + 2, N + 2, name='x')
    v = m.binary_var_matrix(len(K) + 1, N + 2, name='v')
    y = m.binary_var_matrix(N + 2, N + 2, name='y')
    z_A = m.binary_var_matrix(N + 2, len(A) + 1, name='z_A')
    r = m.binary_var_cube(len(A) + 1, N + 2, N + 2, name='r')
    t_KS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_KS')
    t_KE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_KE')
    t_YS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_YS')
    t_YE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_YE')
    t_BS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_BS')
    t_BE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_BE')
    t = m.continuous_var_matrix(1, 1, lb=0, name='t')

    # Objective
    m.minimize(t[0, 0])

    ### Constraints
    m.add_constraints(t[0, 0] >= m.max(t_KE[i, 0], t_YE[i, 0]) for i in J)
    # m.add_constraints(t[0, 0] >= t_KE[i, 0] for i in J)

    ### AGV调度
    m.add_constraints(m.sum(x[k, S, j] for j in J) == 1 for k in K)
    m.add_constraints(m.sum(x[k, i, E] for i in J) == 1 for k in K)
    m.add_constraints(m.sum(x[k, i, j] for i in JS) == m.sum(x[k, j, i] for i in JE) for j in J for k in K)
    m.add_constraints(x[k, i, j] + x[k, j, i] <= 1 for i in J for j in J for k in K)
    m.add_constraints(m.sum(v[k, i] for k in K) == 1 for i in J)
    m.add_constraints(m.sum(x[k, i, j] for j in JE) == v[k, i] for i in J for k in K)
    m.add_constraints(t_KE[i, 0] >= t_KS[i, 0] + TTK[i] + AGV['TQ'] + (v[k, i] - 1) * M1 for i in J for k in K)
    # m.add_constraints(t_KE[i, 0] >= t_KS[i, 0] + TTK[i] + AGV['TQ'] + (v[k, i] - 1) * M1 for i in J for k in K)
    m.add_constraints(t_KS[j, 0] >= t_KE[i, 0] + TK[i][j] + (x[k, i, j] - 1) * M1 for i in J for j in J for k in K)

    ### ASC调度
    m.add_constraint(m.sum(y[S, i] for i in J) == 1)
    m.add_constraint(m.sum(y[i, E] for i in J) == 1)
    m.add_constraints(m.sum(y[i, j] for j in JE) == 1 for i in J)
    m.add_constraints(m.sum(y[i, j] for j in JE) == m.sum(y[j, i] for j in JS) for i in J)
    m.add_constraints(t_YE[i, 0] >= t_YS[i, 0] + 3 * ASC['TY'] + 2 * ASC['BY'] + TTY[i] for i in J)
    m.add_constraints(t_YS[j, 0] >= t_YE[i, 0] + TY[i][j] + (y[i, j] - 1) * M2 for i in J for j in J)

    ## AGV-ASC直接交互联结约束
    m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] >= t_KE[i, 0] + (z_A[i, a] - 1) * M3 for i in JI for a in A)
    m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] <= t_KE[i, 0] + (1 - z_A[i, a]) * M3 for i in JI for a in A)
    m.add_constraints(t_KS[i, 0] >= t_YE[i, 0] + (z_A[i, a] - 1) * M3 for i in JO for a in A)
    m.add_constraints(t_KS[i, 0] <= t_YE[i, 0] + (1 - z_A[i, a]) * M3 for i in JO for a in A)
    # m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] == t_KE[i, 0] for i in JI)
    # m.add_constraints(t_KS[i, 0] == t_YE[i, 0] for i in JO)


    ### 箱子进A区(直接交互位分配约束)
    m.add_constraints(m.sum(z_A[i, a] for a in A) == 1 for i in J)

    ### 箱子进出A区(直接交互)
    m.add_constraints(m.sum(r[a, S, j] for j in J) == 1 for a in A)
    m.add_constraints(m.sum(r[a, i, E] for i in J) == 1 for a in A)
    m.add_constraints(m.sum(r[a, i, j] for i in JS) == m.sum(r[a, j, i] for i in JE) for j in J for a in A)
    m.add_constraints(m.sum(r[a, i, j] for j in JE) == z_A[i, a] for i in J for a in A)
    m.add_constraints(r[a, i, i] == 0 for i in J for a in A)

    m.add_constraints(t_YE[j, 0] - ASC['BY'] >= t_YS[i, 0] + ASC['TY'] + ASC['BY'] + (r[a, i, j] - 1) * M3 for i in JI for j in JO for a in A)
    m.add_constraints(t_KS[j, 0] + AGV['TQ'] + TTK[j] >= t_KE[i, 0] + (r[a, i, j] - 1) * M3 for i in JI for j in JI for a in A)
    m.add_constraints(t_YE[j, 0] - ASC['BY'] >= t_YE[i, 0] + (r[a, i, j] - 1) * M3 for i in JO for j in JO for a in A)
    m.add_constraints(t_KS[j, 0] + AGV['TQ'] + TTK[j] >= t_KS[i, 0] + (r[a, i, j] - 1) * M3 for i in JO for j in JI for a in A)

    
    m.parameters.timelimit = TimeLimit            # 最大求解时间
    m.parameters.mip.tolerances.mipgap = Gap      # best integer 与 best bound之间的GAP
    solution = m.solve(log_output=False)
    m.export(mDir + mDir_solution, 'Model_A_AGV-ASC')  # 输出线性模型文件，保存路径(self.mDir)下
    t_finish = time.time()
    
    '''
    if solution:
        print('/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/ successfully solved /*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/ ')
        print('计算时间%s' % (time.time() - t_start))
        print('********* AGV调度 ********* [k, i, j]')
        for k in K:
            for i in range(N+2):
                for j in range(N+2):
                    if x[k, i, j].solution_value >= 0.5:
                        print((k, i, j))

        print('************* AGV调度时序 *********** [i, KS, KE]')
        for i in J:
            if t_KS[i, 0].solution_value >= 0 and t_KE[i, 0].solution_value >= 0:
                print('[%s,%3.f,%3.f]' % (i, t_KS[i, 0].solution_value, t_KE[i, 0].solution_value))

        print('**************** 起重机调度时序 ************** [i, YS, YE]')
        for i in J:
            if t_YS[i, 0].solution_value >= 0 and t_YE[i, 0].solution_value >= 0:
                print('[%s,%3.f,%3.f]' % (i, t_YS[i, 0].solution_value, t_YE[i, 0].solution_value))

        print('**** ASC 作业顺序 ***** [i, j]')
        for i in range(N+2):
            for j in range(N+2):
                if y[i, j].solution_value >= 0.5:
                    print((i, j))

        print('**** A区：直接交互 ***** [交互位, 任务]')
        for i in range(N+2):
            for a in A:
                if z_A[i, a].solution_value >= 0.5:
                    print((a, i))


        print('**** 任务进出直接交互位置顺序 ***** r[a, i, j]')
        for a in A:
            for i in range(N+2):
                for j in range(N+2):
                    if r[a, i, j].solution_value >= 0.5:
                        print((a, i, j))
    '''
    makespan = t[0, 0].solution_value
    cpu = t_finish - t_start
    # print('model_A_obj %.2f' % makespan)
    # print('model_A_CPU %.3f' % cpu)

    return tour_K, tour_Y, makespan, cpu

In [24]:
################################# 直接交付 #######################################
for i in range(6, 9):
    N = i
    NK = 3   # AGV数量
    NA = 2   # 直接交付位数量
    NB = 0   # 缓冲交付位数量
    
    name = "testing"
    AGV = {'SPD': 4, 'BK': 10, 'TQ': 15}  # AGV参数：水平移动速度、支架上抬/放箱时间、岸桥下作业时间
    ASC = {'SPD': 6, 'BY': 5, 'TY': 10}  # ASC参数：水平移动速度、抓取/释放箱时间、起/落吊时间
    
    Gap = 0         #  max gap
    TimeLimit = 60  # 最大求解时间
    
    J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, dataExp = data_generation(N, NK, NA, NB, name, AGV, ASC)
    tour_K, tour_Y, makespan, cpu = modelASolvebyCplex(N, J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, Gap, TimeLimit) 
    print("N = {:2} | K = {:} | flexible_A = {} | makespan {:7.2f} units | {:7.3f} secs".format(N, NK, NA, makespan, cpu))

N =  6 | K = 3 | flexible_A = 2 | makespan  708.50 units |   1.434 secs
N =  7 | K = 3 | flexible_A = 2 | makespan  804.00 units |   7.020 secs
N =  8 | K = 3 | flexible_A = 2 | makespan  886.00 units |  35.141 secs


## 2.2 基于缓冲位模式下集成调度优化模型 [M2]

In [25]:
def modelBSolvebyCplex(N, J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, Gap, TimeLimit):
    tour_K, tour_Y = [], []
    t_start = time.time()
    
    # define Model
    m = Model("AGV_ASC_flexibleInteractionScheduling")
    S = 0
    E = N + 1
    M1, M2, M3, M4 = 1e+4, 1e+4, 1e+5, 1e+5

    # define Variables
    x = m.binary_var_cube(len(K) + 1, N + 2, N + 2, name='x')
    v = m.binary_var_matrix(len(K) + 1, N + 2, name='v')
    y = m.binary_var_matrix(N + 2, N + 2, name='y')
    z_B = m.binary_var_matrix(N + 2, len(B) + 1, name='z_B')
    w = m.binary_var_cube(len(B) + 1, N + 2, N + 2, name='w')
    t_KS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_KS')
    t_KE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_KE')
    t_YS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_YS')
    t_YE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_YE')
    t_BS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_BS')
    t_BE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_BE')
    t = m.continuous_var_matrix(1, 1, lb=0, name='t')

    # Objective
    m.minimize(t[0, 0])

    ### Constraints
    m.add_constraints(t[0, 0] >= m.max(t_KE[i, 0], t_YE[i, 0]) for i in J)
    # m.add_constraints(t[0, 0] >= t_KE[i, 0] for i in J)

    ### AGV调度
    m.add_constraints(m.sum(x[k, S, j] for j in J) == 1 for k in K)
    m.add_constraints(m.sum(x[k, i, E] for i in J) == 1 for k in K)
    m.add_constraints(m.sum(x[k, i, j] for i in JS) == m.sum(x[k, j, i] for i in JE) for j in J for k in K)
    m.add_constraints(x[k, i, j] + x[k, j, i] <= 1 for i in J for j in J for k in K)
    m.add_constraints(m.sum(v[k, i] for k in K) == 1 for i in J)
    m.add_constraints(m.sum(x[k, i, j] for j in JE) == v[k, i] for i in J for k in K)
    m.add_constraints(t_KE[i, 0] >= t_KS[i, 0] + TTK[i] + AGV['TQ'] + AGV['BK'] * m.sum(z_B[i, b] for b in B) + (v[k, i] - 1) * M1 for i in J for k in K)
    # m.add_constraints(t_KE[i, 0] >= t_KS[i, 0] + TTK[i] + AGV['TQ'] + (v[k, i] - 1) * M1 for i in J for k in K)
    m.add_constraints(t_KS[j, 0] >= t_KE[i, 0] + TK[i][j] + (x[k, i, j] - 1) * M1 for i in J for j in J for k in K)

    ### ASC调度
    m.add_constraint(m.sum(y[S, i] for i in J) == 1)
    m.add_constraint(m.sum(y[i, E] for i in J) == 1)
    m.add_constraints(m.sum(y[i, j] for j in JE) == 1 for i in J)
    m.add_constraints(m.sum(y[i, j] for j in JE) == m.sum(y[j, i] for j in JS) for i in J)
    m.add_constraints(t_YE[i, 0] >= t_YS[i, 0] + 3 * ASC['TY'] + 2 * ASC['BY'] + TTY[i] for i in J)
    m.add_constraints(t_YS[j, 0] >= t_YE[i, 0] + TY[i][j] + (y[i, j] - 1) * M2 for i in J for j in J)


    ## AGV-ASC通过缓冲区  缓冲位交互联结约束
    m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] >= t_KE[i, 0] + (z_B[i, b] - 1) * M3 for i in JI for b in B)
    m.add_constraints(t_KS[i, 0] >= t_YE[i, 0] + (z_B[i, b] - 1) * M3 for i in JO for b in B)


    ### 箱子进入B区(缓冲位分配约束)
    m.add_constraints(m.sum(z_B[i, b] for b in B) == 1 for i in J)

    ### 箱子进出B区(缓冲位交互)
    m.add_constraints(m.sum(w[b, S, j] for j in J) == 1 for b in B)
    m.add_constraints(m.sum(w[b, i, E] for i in J) == 1 for b in B)
    m.add_constraints(m.sum(w[b, i, j] for i in JS) == m.sum(w[b, j, i] for i in JE) for j in J for b in B)
    m.add_constraints(m.sum(w[b, i, j] for j in JE) == z_B[i, b] for i in J for b in B)
    m.add_constraints(w[b, i, i] == 0 for i in J for b in B)
    m.add_constraints(t_BE[i, 0] >= t_BS[i, 0] + (z_B[i, b] - 1) * M3 for i in J for b in B)
    m.add_constraints(t_BS[j, 0] >= t_BE[i, 0] + (w[b, i, j] - 1) * M3 for i in J for j in J for b in B)

    m.add_constraints(t_BS[i, 0] == t_KE[i, 0] for i in JI)
    m.add_constraints(t_BE[i, 0] == t_YS[i, 0] + ASC['TY'] + ASC['BY'] for i in JI)
    m.add_constraints(t_BS[i, 0] == t_YE[i, 0] for i in JO)
    m.add_constraints(t_BE[i, 0] == t_KS[i, 0] for i in JO)

    m.parameters.timelimit = TimeLimit             # 最大求解时间
    m.parameters.mip.tolerances.mipgap = Gap   # best integer 与 best bound之间的GAP
    solution = m.solve(log_output=False)
    m.export(mDir + mDir_solution, 'Model_B_AGV-ASC')  # 输出线性模型文件，保存路径(self.mDir)下
    t_finish = time.time()

    """
    if solution:
        print('/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/ successfully solved /*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/ ')
        print('计算时间%s' % (time.time() - t_start))
        print('********* AGV调度 ********* [k, i, j]')
        for k in K:
            for i in range(N+2):
                for j in range(N+2):
                    if x[k, i, j].solution_value >= 0.5:
                        print((k, i, j))

        print('************* AGV调度时序 *********** [i, KS, KE]')
        for i in J:
            if t_KS[i, 0].solution_value >= 0 and t_KE[i, 0].solution_value >= 0:
                print('[%s,%3.f,%3.f]' % (i, t_KS[i, 0].solution_value, t_KE[i, 0].solution_value))

        print('**************** 起重机调度时序 ************** [i, YS, YE]')
        for i in J:
            if t_YS[i, 0].solution_value >= 0 and t_YE[i, 0].solution_value >= 0:
                print('[%s,%3.f,%3.f]' % (i, t_YS[i, 0].solution_value, t_YE[i, 0].solution_value))

        print('**** ASC 作业顺序 ***** [i, j]')
        for i in range(N+2):
            for j in range(N+2):
                if y[i, j].solution_value >= 0.5:
                    print((i, j))

        print('**** B区：缓冲交互位 ***** [缓冲位, 任务]')
        for i in range(N + 2):
            for b in B:
                if z_B[i, b].solution_value >= 0.5:
                    print((b, i))

        print('**** 任务进缓冲交互位的顺序 ***** w[b, i, j]')
        for b in B:
            for i in range(N+2):
                for j in range(N+2):
                    if w[b, i, j].solution_value >= 0.5:
                        print((b, i, j))

        print('**************** 缓冲交互位（缓冲区）停留时间 [i, BS, BE]**************')
        for i in J:
            for b in B:
                if t_BS[i, 0].solution_value >= 0 and t_BE[i, 0].solution_value >= 0 and z_B[i, b].solution_value >= 0.5:
                    print('[%s,%3.f,%3.f]' % (i, t_BS[i, 0].solution_value, t_BE[i, 0].solution_value))
    """
    
    makespan = t[0, 0].solution_value
    cpu = t_finish - t_start
    
    # print('model_B_obj %.2f' % makespan)
    # print('model_B_CPU %.3f' % cpu)
    
    return tour_K, tour_Y, makespan, cpu

In [27]:
################################# 软交互 #######################################
for i in range(5, 7):
    N = i
    NK = 3   # AGV数量
    NA = 0   # 硬交互位数量
    NB = 3   # 软交互位数量
    
    name = "testing"
    AGV = {'SPD': 4, 'BK': 10, 'TQ': 15}  # AGV参数：水平移动速度、支架上抬/放箱时间、岸桥下作业时间
    ASC = {'SPD': 6, 'BY': 5, 'TY': 10}  # ASC参数：水平移动速度、抓取/释放箱时间、起/落吊时间
    
    Gap = 0         #  max gap
    TimeLimit = 60  # 最大求解时间
    
    J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, dataExp = data_generation(N, NK, NA, NB, name, AGV, ASC)
    tour_K, tour_Y, makespan, cpu  = modelBSolvebyCplex(N, J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, Gap, TimeLimit)
    print("N = {:2} | K = {:} | flexible_B = {} | makespan {:7.2f} units | {:7.3f} secs".format(N, NK, NB, makespan, cpu))


N =  5 | K = 3 | flexible_B = 3 | makespan  557.50 units |   0.467 secs
N =  6 | K = 3 | flexible_B = 3 | makespan  708.50 units |   1.042 secs


## 2.3 直接和缓冲位交付模式下集成调度优化模型 [M3]

In [5]:
def modelCSolvebyCplex(N, J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, Gap, TimeLimit):
    tour_K, tour_Y = [], []
    t_start = time.time()
    # define Model
    m = Model("AGV_ASC_flexibleInteractionScheduling")
    S = 0
    E = N + 1
    M1, M2, M3, M4 = 1e+4, 1e+4, 1e+5, 1e+5

    # define Variables
    x = m.binary_var_cube(len(K) + 1, N + 2, N + 2, name='x')
    v = m.binary_var_matrix(len(K) + 1, N + 2, name='v')
    y = m.binary_var_matrix(N + 2, N + 2, name='y')
    z_A = m.binary_var_matrix(N + 2, len(A) + 1, name='z_A')
    z_B = m.binary_var_matrix(N + 2, len(B) + 1, name='z_B')
    w = m.binary_var_cube(len(B) + 1, N + 2, N + 2, name='w')
    r = m.binary_var_cube(len(A) + 1, N + 2, N + 2, name='r')

    t_KS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_KS')
    t_KE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_KE')
    t_YS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_YS')
    t_YE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_YE')
    t_BS = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_BS')
    t_BE = m.continuous_var_matrix(N + 2, 1, lb=0, name='t_BE')
    t = m.continuous_var_matrix(1, 1, lb=0, name='t')

    # Objective
    m.minimize(t[0, 0])

    ### Constraints
    m.add_constraints(t[0, 0] >= m.max(t_KE[i, 0], t_YE[i, 0]) for i in J)
    # m.add_constraints(t[0, 0] >= t_KE[i, 0] for i in J)



    ### AGV调度
    m.add_constraints(m.sum(x[k, S, j] for j in J) == 1 for k in K)
    m.add_constraints(m.sum(x[k, i, E] for i in J) == 1 for k in K)
    m.add_constraints(m.sum(x[k, i, j] for i in JS) == m.sum(x[k, j, i] for i in JE) for j in J for k in K)
    m.add_constraints(x[k, i, j] + x[k, j, i] <= 1 for i in J for j in J for k in K)
    m.add_constraints(m.sum(v[k, i] for k in K) == 1 for i in J)
    m.add_constraints(m.sum(x[k, i, j] for j in JE) == v[k, i] for i in J for k in K)
    m.add_constraints(t_KE[i, 0] >= t_KS[i, 0] + TTK[i] + AGV['TQ'] + AGV['BK'] * m.sum(z_B[i, b] for b in B) + (v[k, i] - 1) * M1 for i in J for k in K)
    # m.add_constraints(t_KE[i, 0] >= t_KS[i, 0] + TTK[i] + AGV['TQ'] + (v[k, i] - 1) * M1 for i in J for k in K)
    m.add_constraints(t_KS[j, 0] >= t_KE[i, 0] + TK[i][j] + (x[k, i, j] - 1) * M1 for i in J for j in J for k in K)

    ### ASC调度
    m.add_constraint(m.sum(y[S, i] for i in J) == 1)
    m.add_constraint(m.sum(y[i, E] for i in J) == 1)
    m.add_constraints(m.sum(y[i, j] for j in JE) == 1 for i in J)
    m.add_constraints(m.sum(y[i, j] for j in JE) == m.sum(y[j, i] for j in JS) for i in J)
    m.add_constraints(t_YE[i, 0] >= t_YS[i, 0] + 3 * ASC['TY'] + 2 * ASC['BY'] + TTY[i] for i in J)
    m.add_constraints(t_YS[j, 0] >= t_YE[i, 0] + TY[i][j] + (y[i, j] - 1) * M2 for i in J for j in J)

    ## AGV-ASC硬交互
    m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] >= t_KE[i, 0] + (z_A[i, a] - 1) * M3 for i in JI for a in A)
    m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] <= t_KE[i, 0] + (1 - z_A[i, a]) * M3 for i in JI for a in A)
    m.add_constraints(t_KS[i, 0] >= t_YE[i, 0] + (z_A[i, a] - 1) * M3 for i in JO for a in A)
    m.add_constraints(t_KS[i, 0] <= t_YE[i, 0] + (1 - z_A[i, a]) * M3 for i in JO for a in A)
    # m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] == t_KE[i, 0] for i in JI)
    # m.add_constraints(t_KS[i, 0] == t_YE[i, 0] for i in JO)

    ## AGV-ASC通过缓冲区  软交互
    m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] >= t_KE[i, 0] + (z_B[i, b] - 1) * M3 for i in JI for b in B)
    m.add_constraints(t_KS[i, 0] >= t_YE[i, 0] + (z_B[i, b] - 1) * M3 for i in JO for b in B)

    ### 箱子进A区或B区(硬交互或软交互)
    m.add_constraints(m.sum(z_A[i, a] for a in A) + m.sum(z_B[i, b] for b in B) == 1 for i in J)

    ### 箱子进出A区(硬交互)
    m.add_constraints(m.sum(r[a, S, j] for j in J) == 1 for a in A)
    m.add_constraints(m.sum(r[a, i, E] for i in J) == 1 for a in A)
    m.add_constraints(m.sum(r[a, i, j] for i in JS) == m.sum(r[a, j, i] for i in JE) for j in J for a in A)
    m.add_constraints(m.sum(r[a, i, j] for j in JE) == z_A[i, a] for i in J for a in A)
    m.add_constraints(r[a, i, i] == 0 for i in J for a in A)

    m.add_constraints(t_YE[j, 0] - ASC['BY'] >= t_YS[i, 0] + ASC['TY'] + ASC['BY'] + (r[a, i, j] - 1) * M3 for i in JI for j in JO for a in A)
    m.add_constraints(t_KS[j, 0] + AGV['TQ'] + TTK[j] >= t_KE[i, 0] + (r[a, i, j] - 1) * M3 for i in JI for j in JI for a in A)
    m.add_constraints(t_YE[j, 0] - ASC['BY'] >= t_YE[i, 0] + (r[a, i, j] - 1) * M3 for i in JO for j in JO for a in A)
    m.add_constraints(t_KS[j, 0] + AGV['TQ'] + TTK[j] >= t_KS[i, 0] + (r[a, i, j] - 1) * M3 for i in JO for j in JI for a in A)

    ### 箱子进出B区(软交互)
    m.add_constraints(m.sum(w[b, S, j] for j in J) == 1 for b in B)
    m.add_constraints(m.sum(w[b, i, E] for i in J) == 1 for b in B)
    m.add_constraints(m.sum(w[b, i, j] for i in JS) == m.sum(w[b, j, i] for i in JE) for j in J for b in B)
    m.add_constraints(m.sum(w[b, i, j] for j in JE) == z_B[i, b] for i in J for b in B)
    m.add_constraints(w[b, i, i] == 0 for i in J for b in B)
    m.add_constraints(t_BE[i, 0] >= t_BS[i, 0] + (z_B[i, b] - 1) * M3 for i in J for b in B)
    m.add_constraints(t_BS[j, 0] >= t_BE[i, 0] + (w[b, i, j] - 1) * M3 for i in J for j in J for b in B)

    # m.add_constraints(t_BS[i, 0] == t_KE[i, 0] for i in JI)
    # m.add_constraints(t_BE[i, 0] == t_YS[i, 0] + ASC['TY'] + ASC['BY'] for i in JI)
    # m.add_constraints(t_BS[i, 0] == t_YE[i, 0] for i in JO)
    # m.add_constraints(t_BE[i, 0] == t_KS[i, 0] for i in JO)
    #
    m.add_constraints(t_BS[i, 0] >= t_KE[i, 0] + (z_B[i, b] - 1) * M3 for i in JI for b in B)
    m.add_constraints(t_YS[i, 0] + ASC['TY'] + ASC['BY'] >= t_BE[i, 0] + (z_B[i, b] - 1) * M3 for i in JI for b in B)
    m.add_constraints(t_BS[i, 0] >= t_YE[i, 0] + (z_B[i, b] - 1) * M3 for b in B for i in JO)
    m.add_constraints(t_KS[i, 0] >= t_BE[i, 0] + (z_B[i, b] - 1) * M3 for b in B for i in JO)

    m.parameters.timelimit = TimeLimit  # 最大求解时间
    m.parameters.mip.tolerances.mipgap = Gap  # best integer 与 best bound之间的GAP
    solution = m.solve(log_output=False)
    # m.print_information()
    # print(solution.solve_details)
    m.export(mDir + mDir_solution, 'Model_C_AGV-ASC')  # 输出线性模型文件，保存路径(self.mDir)下
    t_finish = time.time()
 
    """
    if solution:
        print('/*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/ successfully solved /*/*/*/*/*/*/*/*/*/*/*/*/*/*/*/ ')
        # print('计算时间%s' % (time.time() - t_start))
        print('********* AGV调度 ********* [k, i, j]')
        for k in K:
            for i in range(N+2):
                for j in range(N+2):
                    if x[k, i, j].solution_value >= 0.5:
                        print((k, i, j))

        print('************* AGV调度时序 *********** [i, KS, KE]')
        for i in J:
            if t_KS[i, 0].solution_value >= 0 and t_KE[i, 0].solution_value >= 0:
                print('[%s,%3.f,%3.f]' % (i, t_KS[i, 0].solution_value, t_KE[i, 0].solution_value))

        print('**************** 起重机调度时序 ************** [i, YS, YE]')
        for i in J:
            if t_YS[i, 0].solution_value >= 0 and t_YE[i, 0].solution_value >= 0:
                print('[%s,%3.f,%3.f]' % (i, t_YS[i, 0].solution_value, t_YE[i, 0].solution_value))

        print('**** ASC 作业顺序 ***** [i, j]')
        for i in range(N+2):
            for j in range(N+2):
                if y[i, j].solution_value >= 0.5:
                    print((i, j))

        print('**** A区：硬交互 ***** [位, 任务]')
        for i in range(N + 2):
            for a in A:
                if z_A[i, a].solution_value >= 0.5:
                    print((a, i))

        print('**** B区：软交互 ***** [缓冲位, 任务]')
        for i in range(N + 2):
            for b in B:
                if z_B[i, b].solution_value >= 0.5:
                    print((b, i))

        print('**** 任务进软交互位的顺序 ***** w[b, i, j]')
        for b in B:
            for i in range(N+2):
                for j in range(N+2):
                    if w[b, i, j].solution_value >= 0.5:
                        print((b, i, j))

        print('**************** 软交互位（缓冲区）停留时间 [i, BS, BE]**************')
        for i in J:
            for b in B:
                if t_BS[i, 0].solution_value >= 0 and t_BE[i, 0].solution_value >= 0 and z_B[i, b].solution_value >= 0.5:
                    print('[%s,%3.f,%3.f]' % (i, t_BS[i, 0].solution_value, t_BE[i, 0].solution_value))
    """


    makespan = t[0, 0].solution_value
    cpu = t_finish - t_start
    # print('model_C_obj %.2f' % makespan)
    # print('model_C_CPU %.3f' % cpu)

    return tour_K, tour_Y, makespan, cpu





In [6]:
################################# 直接和缓冲位交付 #######################################

NM = [6, 7]    # 任务数量
KM = [3, 4]   # AGV数量
# NAB = {1:(5, 3), 2:(4, 3), 3:(3, 3), 4: (3, 5), 5: (3, 4)} # (硬,软）交互位数量
NAB = {1:(2, 3)} # (硬,软）交互位数量

name = "testing"
AGV = {'SPD': 4, 'BK': 10, 'TQ': 15}  # AGV参数：水平移动速度、支架上抬/放箱时间、岸桥下作业时间
ASC = {'SPD': 6, 'BY': 5, 'TY': 10}  # ASC参数：水平移动速度、抓取/释放箱时间、起/落吊时间

Gap = 0           # 最大GAP
TimeLimit = 3600    # 最大计算时间

for NK in KM:
    for N in NM:
        for ab in NAB.keys():
            NA = NAB[ab][0]
            NB = NAB[ab][1]
            J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, dataExp = data_generation(N, NK, NA, NB, name, AGV, ASC)
            tour_K, tour_Y, makespan, cpu = modelCSolvebyCplex(N, J, JI, JO, JS, JE, JD, K, B, A, TTK, TTY, TK, TY, AGV, ASC, Gap, TimeLimit) # 柔性交互
            print("N = {:2} | K = {:} | flexible = {} | makespan {:7.2f} units | {:7.3f} secs".format(N, NK, NAB[ab], makespan,cpu))

N =  6 | K = 3 | flexible = (2, 3) | makespan  708.50 units |   2.472 secs
N =  7 | K = 3 | flexible = (2, 3) | makespan  800.50 units |  32.097 secs
N =  6 | K = 4 | flexible = (2, 3) | makespan  708.50 units |   3.566 secs
N =  7 | K = 4 | flexible = (2, 3) | makespan  800.50 units |  11.152 secs
